In [28]:
import csv

from sklearn.kernel_ridge import KernelRidge
import numpy as np
from sklearn.metrics import mean_squared_error
import time
import warnings
warnings.filterwarnings('ignore')

TEST_DATA_FILE_1 = "test_data_1.csv"
TEST_DATA_FILE_2 = "test_data_2.csv"
TRAIN_DATA_FILE_1 = "train_data_1.csv"
TRAIN_DATA_FILE_2 = "train_data_2.csv"
PROD_INDEX = 6
DIR_INDEX = 5
ERROR_THRESHOLD_1 = 0
ERROR_THRESHOLD_2 = 0.2
ERROR_THRESHOLD_3 = 0.4

In [29]:
def read_data_file(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0]))
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

In [30]:
train_data_1_labels, train_data_1_values = read_data_file(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [31]:
def get_error_metric(threshold,actual_labels,predicted_labels):
    error_count = 0
    errors = np.zeros(len(actual_labels))
    for index in range(0,len(actual_labels)):
        errors[index] = (abs(float(actual_labels[index])-float(predicted_labels[index])))
        if abs(float(actual_labels[index])-float(predicted_labels[index])) > threshold:
            error_count +=1
    return error_count*100/len(actual_labels),np.mean(errors),np.std(errors)

def get_mean_squared_error(actual_labels,predicted_labels):
    return mean_squared_error(actual_labels,predicted_labels)

In [32]:
def get_predictions(classifier,val,data):
    if val==1:
        X = np.asarray(train_data_1_values)
        Y = np.asarray(train_data_1_labels)
    else:
        X = np.asarray(train_data_2_values)
        Y = np.asarray(train_data_2_labels)
    classifier.fit(X,Y)
    return classifier.predict(data)

In [33]:
classifier = KernelRidge()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:1
Errors for Mechanism-1: Zero Tolerance: 100 0.289249321055 0.302990517526
Errors for Mechanism-1: 0.1 Tolerance: 49 0.289249321055 0.302990517526
Errors for Mechanism-1: 0.2 Tolerance: 26 0.289249321055 0.302990517526
MSE Mechansim-1: 0.175468423441


In [34]:
classifier = KernelRidge(alpha=0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

t = time.time()
print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

t2= time.time()
print 'Time:',t2-t
classifier = KernelRidge(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Time:',time.time()-t2

Lambda:0.1
Errors for Mechanism-1: Zero Tolerance: 100 0.28924278387 0.302998897057
Errors for Mechanism-1: 0.1 Tolerance: 49 0.28924278387 0.302998897057
Errors for Mechanism-1: 0.2 Tolerance: 26 0.28924278387 0.302998897057
MSE Mechansim-1: 0.175469719639
Time: 0.039999961853
Lambda:-1
Errors for Mechanism-1: Zero Tolerance: 100 0.28923478538 0.303009181934
Errors for Mechanism-1: 0.1 Tolerance: 49 0.28923478538 0.303009181934
Errors for Mechanism-1: 0.2 Tolerance: 26 0.28923478538 0.303009181934
MSE Mechansim-1: 0.17547132541
Time: 659.506000042


In [35]:
classifier = KernelRidge(alpha=0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

t= time.time()
print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

t2= time.time()
print 'Time:',t2-t
classifier = KernelRidge(alpha=2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Time:',time.time()-t2

Lambda:0.01
Errors for Mechanism-1: Zero Tolerance: 100 0.289242129917 0.302999736651
Errors for Mechanism-1: 0.1 Tolerance: 49 0.289242129917 0.302999736651
Errors for Mechanism-1: 0.2 Tolerance: 26 0.289242129917 0.302999736651
MSE Mechansim-1: 0.17546985013
Time: 0.0279998779297
Lambda:2
Errors for Mechanism-1: Zero Tolerance: 100 0.289256577184 0.302981244177
Errors for Mechanism-1: 0.1 Tolerance: 49 0.289256577184 0.302981244177
Errors for Mechanism-1: 0.2 Tolerance: 26 0.289256577184 0.302981244177
MSE Mechansim-1: 0.175467001767
Time: 13.9330000877


In [36]:
classifier = KernelRidge(alpha=-2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

t = time.time()
print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Time taken:',time.time()-t

Lambda:-2
Errors for Mechanism-1: Zero Tolerance: 100 0.289227505801 0.303018573192
Errors for Mechanism-1: 0.1 Tolerance: 49 0.289227505801 0.303018573192
Errors for Mechanism-1: 0.2 Tolerance: 26 0.289227505801 0.303018573192
MSE Mechansim-1: 0.175472805811
Time taken: 0.0720000267029


In [ ]:
classifier = KernelRidge()
obs_labels = get_predictions(classifier,2,test_data_2_values)

t= time.time()
print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Time taken:',time.time()-t

Lambda:1
Errors for Mechanism-2: Zero Tolerance: 100 2.82416575643 1.77590259241
Errors for Mechanism-2: 0.1 Tolerance: 96 2.82416575643 1.77590259241
Errors for Mechanism-2: 0.2 Tolerance: 93 2.82416575643 1.77590259241
MSE Mechansim-2: 11.1297422375
Time taken: 0.0199999809265


In [ ]:
classifier = KernelRidge(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)
t = time.time()
print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Time taken',time.time()-t

In [13]:
classifier = KernelRidge(alpha=2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

t = time.time()
print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels) 
print 'Time taken',time.time() - t

classifier = KernelRidge(alpha=-2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

t2 = time.time()
print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Time taken:',time.time()-t2

Lambda:2
Errors for Mechanism-2: Zero Tolerance: 100 2.82428205474 1.77570205565
Errors for Mechanism-2: 0.1 Tolerance: 96 2.82428205474 1.77570205565
Errors for Mechanism-2: 0.2 Tolerance: 93 2.82428205474 1.77570205565
MSE Mechansim-2: 11.1296869152
Time taken 0.0450000762939
Lambda:-2
Errors for Mechanism-2: Zero Tolerance: 100 2.82381796075 1.77650658334
Errors for Mechanism-2: 0.1 Tolerance: 96 2.82381796075 1.77650658334
Errors for Mechanism-2: 0.2 Tolerance: 93 2.82381796075 1.77650658334
MSE Mechansim-2: 11.1299235161


In [14]:

avg_rating = 6.031

def read_data_file_2(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0])-avg_rating)
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

train_data_1_labels, train_data_1_values = read_data_file_2(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file_2(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file_2(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file_2(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [15]:
classifier = KernelRidge()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)


Lambda:1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.71258363701 0.636740935765
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.71258363701 0.636740935765
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.71258363701 0.636740935765
(2) MSE Mechansim-1: 0.913214459013


In [16]:
classifier = KernelRidge(alpha=0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = KernelRidge(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:0.01
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.712569730374 0.636761129028
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.712569730374 0.636761129028
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.712569730374 0.636761129028
(2) MSE Mechansim-1: 0.913220356087
Lambda:-1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.712555532932 0.636781803319
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.712555532932 0.636781803319
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.712555532932 0.636781803319
(2) MSE Mechansim-1: 0.91322645255


In [17]:
classifier = KernelRidge(alpha=0.001)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.001'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = KernelRidge(alpha=2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)


Lambda:0.001
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.712569607512 0.636761309748
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.712569607512 0.636761309748
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.712569607512 0.636761309748
(2) MSE Mechansim-1: 0.913220411142
Lambda:2
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.712597672814 0.636720611457
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.712597672814 0.636720611457
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.712597672814 0.636720611457
(2) MSE Mechansim-1: 0.913208580354


In [18]:
classifier = KernelRidge(alpha=-2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:-2
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.712541464658 0.636802346856
Errors(2) for Mechanism-1: 0.1 Tolerance: 80 0.712541464658 0.636802346856
Errors(2) for Mechanism-1: 0.2 Tolerance: 62 0.712541464658 0.636802346856
(2) MSE Mechansim-1: 0.913232567818


In [19]:
classifier = KernelRidge()
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070279333 0.288348931849
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070279333 0.288348931849
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070279333 0.288348931849
(2) MSE Mechansim-2: 0.176212981831


In [20]:
classifier = KernelRidge(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = KernelRidge(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = KernelRidge(alpha=0.01)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)


Lambda:0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070481677 0.288346887045
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070481677 0.288346887045
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070481677 0.288346887045
(2) MSE Mechansim-2: 0.17621192606
Lambda:-1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070724425 0.288344427576
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070724425 0.288344427576
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070724425 0.288344427576
(2) MSE Mechansim-2: 0.176210655816
Lambda:0.01
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070501727 0.288346684173
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070501727 0.288346684173
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070501727 0.288346684173
(2) MSE Mechansim-2: 0.176211821297


In [21]:
classifier = KernelRidge(alpha=2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = KernelRidge(alpha=0.02)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)



Lambda:2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070102441 0.288351183118
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070102441 0.288351183118
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070102441 0.288351183118
(2) MSE Mechansim-2: 0.176214172209
Lambda:2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070499502 0.288346706698
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070499502 0.288346706698
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070499502 0.288346706698
(2) MSE Mechansim-2: 0.17621183293


In [22]:
classifier = KernelRidge(alpha=-2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:-2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070940733 0.288342229832
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070940733 0.288342229832
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070940733 0.288342229832
(2) MSE Mechansim-2: 0.176209520384


In [23]:
classifier = KernelRidge(alpha=0.5)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.5'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.5
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070392161 0.288347792244
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070392161 0.288347792244
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070392161 0.288347792244
(2) MSE Mechansim-2: 0.176212393465


In [24]:
classifier = KernelRidge(alpha=0.2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.30507045936 0.288347112804
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.30507045936 0.288347112804
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.30507045936 0.288347112804
(2) MSE Mechansim-2: 0.176212042637


In [25]:
classifier = KernelRidge(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070481677 0.288346887045
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070481677 0.288346887045
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070481677 0.288346887045
(2) MSE Mechansim-2: 0.17621192606


In [26]:
classifier = KernelRidge(alpha=0.02)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.02'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.02
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070499502 0.288346706698
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070499502 0.288346706698
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070499502 0.288346706698
(2) MSE Mechansim-2: 0.17621183293


In [27]:
classifier = KernelRidge(alpha=0.002)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.002'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.002
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.305070503498 0.288346666163
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.305070503498 0.288346666163
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.305070503498 0.288346666163
(2) MSE Mechansim-2: 0.176211811992
